In [34]:
import random

class Expression:
    @classmethod
    def create_from(cls, input) -> 'Expression':
        if isinstance(input, cls):
            return input
        return Expression(input)
    
    def __init__(self, num, expr=None):
        self.number = num
        if expr is None:
            expr = (0, str(num))
        
        self.num_operations = expr[0]
        self.expression_string = expr[1]

    def __eq__(self, other):
        return (
            self.number == other.number 
            and self.num_operations == other.num_operations
            and self.expression_string == other.expression_string
        )
    
    def __repr__(self):
        return f"|{self.expression_string} = {self.number} ({self.num_operations} ops)|"

class ExprMultiset(dict):
    @classmethod
    def from_list(cls, nums: list) -> 'ExprMultiset':
        obj = cls()
        for num in nums:
            obj.add(num)
        return obj
    
    @property
    def counts(self):
        return {num: self.count(num) for num in self} 
    
    @property
    def expression_list(self):
        return [expr for num in self for expr in self.expressions(num)]
    
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

    def __len__(self):
        return len(self.expression_list)

    def expressions(self, num):
        return [expr for exprs in self[num].values() for expr in exprs]

    def count(self, num):
        return len(self.expressions(num))
    
    def random_choice(self, replacement=False):
        expression = random.choice(self.expression_list)
        if not replacement:
            self.remove(expression)
        return expression

    def add(self, expression):
        expression = Expression.create_from(expression)
        
        num = expression.number
        num_ops = expression.num_operations

        if num not in self:
            self[num] = {num_ops: [expression]}
        elif num_ops not in self[num]:
            self[num][num_ops] = [expression]
        else:
            self[num][num_ops].append(expression)

    def remove(self, expression):
        expression = Expression.create_from(expression)
        
        num = expression.number
        num_ops = expression.num_operations

        if (num not in self) or (num_ops not in self[num]) or (expression not in self[num][num_ops]):
            raise ValueError(f"Expression {expression} not found in multiset")
        self[num][num_ops].remove(expression)
        
        if not self[num][num_ops]:
            del self[num][num_ops]
        if not self[num]:
            del self[num]

example = ExprMultiset.from_list([1, 2, 2, 2, 2, 3])
example.add(1)
example.remove(2)
try:
    print("Attempting to remove unfound element")
    example.remove(5)
except ValueError as e:
    print(e)
    
print("Random element: ", example.random_choice(replacement=True))
example.counts, example.expression_list

Attempting to remove unfound element
Expression |5 = 5 (0 ops)| not found in multiset
Random element:  |2 = 2 (0 ops)|


({1: 2, 2: 3, 3: 1},
 [|1 = 1 (0 ops)|,
  |1 = 1 (0 ops)|,
  |2 = 2 (0 ops)|,
  |2 = 2 (0 ops)|,
  |2 = 2 (0 ops)|,
  |3 = 3 (0 ops)|])

In [35]:
def operate(*args):
    if len(args) != 2:
        raise ValueError("Must pass in exactly two arguments")
    a, b = args[0], args[1]
    a = Expression.create_from(a)
    b = Expression.create_from(b)
    num_ops = a.num_operations + b.num_operations + 1
    
    a, a_expr_str = a.number, a.expression_string
    b, b_expr_str = b.number, b.expression_string
    
    results = [
        Expression(a + b, (num_ops, f"({a_expr_str} + {b_expr_str})")),
        Expression(a - b, (num_ops, f"({a_expr_str} - {b_expr_str})")),
        Expression(b - a, (num_ops, f"({b_expr_str} - {a_expr_str})")),
        Expression(a * b, (num_ops, f"({a_expr_str} * {b_expr_str})"))
    ]

    if a != 0: 
        results.append(Expression(b / a, (num_ops, f"({b_expr_str} / {a_expr_str})")))

    if b != 0:
        results.append(Expression(a / b, (num_ops, f"({a_expr_str} / {b_expr_str})")))

    return results

operate(4, 2)

[|(4 + 2) = 6 (1 ops)|,
 |(4 - 2) = 2 (1 ops)|,
 |(2 - 4) = -2 (1 ops)|,
 |(4 * 2) = 8 (1 ops)|,
 |(2 / 4) = 0.5 (1 ops)|,
 |(4 / 2) = 2.0 (1 ops)|]

In [36]:
from copy import copy, deepcopy
import numpy as np

class Solver:
    def __init__(self, operate_fn=operate):
        self.binary_operate = operate_fn

    def gen_expressions(self, nums, goal_operations=3, target=None):
        exprs = ExprMultiset.from_list(nums)
        final_expressions = []
        correct_expressions = []

        def gen_expressions_rec(exprs):
            if len(exprs) <= 1:
                return
            num_exprs = len(exprs.expression_list)
            for i in range(len(exprs)):
                for j in range(i):
                    exprs_copy = deepcopy(exprs)
                    a, b = exprs_copy.expression_list[i], exprs_copy.expression_list[j]
                    exprs_copy.remove(a)
                    exprs_copy.remove(b)
                    result_exprs = self.binary_operate(a, b)
                    for expr in result_exprs:
                        if expr.num_operations == goal_operations:
                            final_expressions.append(expr)
                            if (target is not None) and np.abs(expr.number - target) < 1e-6:
                                correct_expressions.append(expr)
                        new_exprs = deepcopy(exprs_copy)
                        new_exprs.add(expr)
                        gen_expressions_rec(new_exprs)

        gen_expressions_rec(exprs)

        return final_expressions, correct_expressions

s = Solver()
final_expressions, correct_expressions = s.gen_expressions([1, 3, 4, 6], target=24)

len(final_expressions)

KeyboardInterrupt: 

In [ ]:
correct_expressions

[|(6 / (1 - (3 / 4))) = 24.0 (3 ops)|]